# Convolution GRU (ConvGRU) Model for Calculating the Intensity of Tropical Cyclones

In [1]:
import tensorflow as tf
print("Available devices:", tf.config.list_physical_devices())


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Building the ConvGRU Model

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Define the ConvGRU2DLayer (same as before)
class ConvGRU2DLayer(layers.Layer):
    def __init__(self, filters, kernel_size, return_sequences=True, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.return_sequences = return_sequences

    def build(self, input_shape):
        self.input_projection = layers.Conv2D(self.filters, (1, 1), padding="same")
        self.conv_z = layers.Conv2D(self.filters, self.kernel_size, padding="same", activation="sigmoid")
        self.conv_r = layers.Conv2D(self.filters, self.kernel_size, padding="same", activation="sigmoid")
        self.conv_h = layers.Conv2D(self.filters, self.kernel_size, padding="same", activation="tanh")
        super().build(input_shape)

    def call(self, inputs):
        batch_size, time_steps, height, width, channels = tf.unstack(tf.shape(inputs))
        time_steps=inputs.shape[1]
        h_t = tf.zeros((batch_size, height, width, self.filters))
        outputs = []

        for t in range(time_steps):
            x_t = inputs[:, t, :, :, :]
            x_projected = self.input_projection(x_t)
            z = self.conv_z(x_projected)+self.conv_z(h_t)
            r = self.conv_r(x_projected)+self.conv_z(h_t)
            h_tilde = self.conv_h(r * h_t)
            h_t = (1 - z) * h_t + z * h_tilde

            if self.return_sequences:
                outputs.append(h_t)

        if self.return_sequences:
            outputs = tf.stack(outputs, axis=1)
        else:
            outputs = h_t

        return outputs

# Define the model (same as before)
def build_convgru_model(input_shape=(8, 95, 95, 2)):
    input_tensor = layers.Input(shape=input_shape)
    x = ConvGRU2DLayer(filters=32, kernel_size=(3, 3), return_sequences=True)(input_tensor)
    x = layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(4, 3, 3), padding='same')(x)
    x = ConvGRU2DLayer(filters=64, kernel_size=(3, 3), return_sequences=True)(x)
    x = layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(4, 3, 3), padding='same')(x)
    x = ConvGRU2DLayer(filters=128, kernel_size=(3, 3), return_sequences=True)(x)
    x = layers.Conv3D(filters=128, kernel_size=(3, 3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    x = layers.Flatten()(x)
    model = models.Model(inputs=input_tensor, outputs=x)
    return model
input_shape = (8, 95, 95, 2)  # (time_steps, features)
convgru_model = build_convgru_model(input_shape=input_shape)
convgru_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 8, 95, 95, 2)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_gru2d_layer (ConvGRU2DLayer)    │ (None, 8, 95, 95, 32)       │          27,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d (Conv3D)                      │ (None, 8, 95, 95, 32)       │          27,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 2, 32, 32, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_gru2d_layer_1 (ConvGRU2DLayer)  │ (None, 2, 32, 32, 64)       │         112,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 2, 32, 32, 64)       │         110,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 1, 11, 11, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv_gru2d_layer_2 (ConvGRU2DLayer)  │ (None, 1, 11, 11, 128)      │         451,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 1, 11, 11, 128)      │         442,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 1, 6, 6, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,172,640 (4.47 MB)

 Trainable params: 1,172,640 (4.47 MB)

 Non-trainable params: 0 (0.00 B)

# Building the Radial Structure

In [3]:
def radial_structure_subnet(input_shape):
    """
    Creates the subnet for extracting TC radial structure features using a five-branch CNN design with 2D convolutions.

    Parameters:
    - input_shape: tuple, shape of the input data (e.g., (95, 95, 3))

    Returns:
    - model: tf.keras.Model, the radial structure subnet model
    """
    
    input_tensor = layers.Input(shape=input_shape)

    # Divide input data into four quadrants (NW, NE, SW, SE)
    # Assuming the input shape is (batch_size, height, width, channels)
    
    # Quadrant extraction - using slicing to separate quadrants
    nw_quadrant = input_tensor[:, :input_shape[0]//2, :input_shape[1]//2, :]
    ne_quadrant = input_tensor[:, :input_shape[0]//2, input_shape[1]//2:, :]
    sw_quadrant = input_tensor[:, input_shape[0]//2:, :input_shape[1]//2, :]
    se_quadrant = input_tensor[:, input_shape[0]//2:, input_shape[1]//2:, :]


    target_height = max(input_shape[0]//2, input_shape[0] - input_shape[0]//2)  # 48
    target_width = max(input_shape[1]//2, input_shape[1] - input_shape[1]//2)  # 48
    
    # Padding the quadrants to match the target size (48, 48)
    nw_quadrant = layers.ZeroPadding2D(padding=((0, target_height - nw_quadrant.shape[1]), 
                                                (0, target_width - nw_quadrant.shape[2])))(nw_quadrant)
    ne_quadrant = layers.ZeroPadding2D(padding=((0, target_height - ne_quadrant.shape[1]), 
                                                (0, target_width - ne_quadrant.shape[2])))(ne_quadrant)
    sw_quadrant = layers.ZeroPadding2D(padding=((0, target_height - sw_quadrant.shape[1]), 
                                                (0, target_width - sw_quadrant.shape[2])))(sw_quadrant)
    se_quadrant = layers.ZeroPadding2D(padding=((0, target_height - se_quadrant.shape[1]), 
                                                (0, target_width - se_quadrant.shape[2])))(se_quadrant)

    print(nw_quadrant.shape)
    print(ne_quadrant.shape)
    print(sw_quadrant.shape)
    print(se_quadrant.shape)
    # Main branch (processing the entire structure)
    main_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(input_tensor)
    y=layers.MaxPool2D()(main_branch)

    y = layers.ZeroPadding2D(padding=((0, target_height - y.shape[1]), 
                                   (0, target_width - y.shape[2])))(y)
    # Side branches (processing the individual quadrants)
    nw_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(nw_quadrant)
    ne_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(ne_quadrant)
    sw_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(sw_quadrant)
    se_branch = layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu')(se_quadrant)
    
    # Apply padding to the side branches to match the dimensions of the main branch
    # nw_branch = layers.UpSampling2D(size=(2, 2), interpolation='nearest')(nw_branch)
    # ne_branch = layers.UpSampling2D(size=(2, 2), interpolation='nearest')(ne_branch)
    # sw_branch = layers.UpSampling2D(size=(2, 2), interpolation='nearest')(sw_branch)
    # se_branch = layers.UpSampling2D(size=(2, 2), interpolation='nearest')(se_branch)
    
    # Fusion operations (concatenate the outputs from the main branch and side branches)
    fusion = layers.concatenate([y, nw_branch, ne_branch, sw_branch, se_branch], axis=-1)
    
    # Additional convolution layer to combine the fused features
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(fusion)
    x=layers.MaxPool2D(pool_size=(2, 2))(x)
    # Final dense layer for further processing
    nw_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(nw_branch)
    
    ne_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(ne_branch)
    sw_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(sw_branch)
    se_branch = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(se_branch)
    nw_branch = layers.MaxPool2D(pool_size=(2, 2))(nw_branch)
    ne_branch = layers.MaxPool2D(pool_size=(2, 2))(ne_branch)
    sw_branch = layers.MaxPool2D(pool_size=(2, 2))(sw_branch)
    se_branch = layers.MaxPool2D(pool_size=(2, 2))(se_branch)

    fusion = layers.concatenate([x, nw_branch, ne_branch, sw_branch, se_branch], axis=-1)
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(fusion)
    x=layers.MaxPool2D(pool_size=(2, 2))(x)
    
    nw_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(nw_branch)
    
    ne_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(ne_branch)
    sw_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(sw_branch)
    se_branch = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(se_branch)
    nw_branch = layers.MaxPool2D(pool_size=(2, 2))(nw_branch)
    ne_branch = layers.MaxPool2D(pool_size=(2, 2))(ne_branch)
    sw_branch = layers.MaxPool2D(pool_size=(2, 2))(sw_branch)
    se_branch = layers.MaxPool2D(pool_size=(2, 2))(se_branch)

    fusion = layers.concatenate([x, nw_branch, ne_branch, sw_branch, se_branch], axis=-1)
    x = layers.Conv2D(filters=32, kernel_size=(3, 3),  activation='relu')(fusion)
    x=layers.Conv2D(filters=32, kernel_size=(3, 3), activation=None)(x)
    # Create and return the model
    x=layers.Flatten()(x)
    model = models.Model(inputs=input_tensor, outputs=x)
    return model

# Define input shape (batch_size, height, width, channels)
# input_shape = (95, 95, 8)  # Example input shape (95x95 spatial resolution, 3 channels)

# # Build the model
# model = radial_structure_subnet(input_shape)

# # Model summary
# model.summary()


# Building the CNN Model

In [4]:
def build_cnn_model(input_shape=(8, 8, 1)):
    # Define the input layer
    input_tensor = layers.Input(shape=input_shape)
    
    # Convolutional layer
    x = layers.Conv2D(64, (3, 3), padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Flatten layer
    x = layers.Flatten()(x)
    
    # Create the model
    model = models.Model(inputs=input_tensor, outputs=x)
    
    return model

# # Example usage:
# cnn_model = build_cnn_model(input_shape=(8, 8, 1))
# cnn_model.summary()


# Building the Combined Model

In [5]:
from tensorflow.keras import layers, models, Input

def build_combined_model():
    # Define input shapes
    input_shape_3d = (8, 95, 95, 2)
    input_shape_radial = (95, 95, 8)
    input_shape_cnn = (8, 8, 1)
    
    input_shape_latitude = (8,)
    input_shape_longitude = (8,)
    input_shape_other = (9,)

    # Build individual models
    model_3d = build_convgru_model(input_shape=input_shape_3d)
    model_radial = radial_structure_subnet(input_shape=input_shape_radial)
    model_cnn = build_cnn_model(input_shape=input_shape_cnn)

    # Define new inputs
    input_latitude = Input(shape=input_shape_latitude ,name="latitude_input")
    input_longitude = Input(shape=input_shape_longitude, name="longitude_input")
    input_other = Input(shape=input_shape_other, name="other_input")

    # Flatten the additional inputs
    flat_latitude = layers.Dense(32,activation='relu')(input_latitude)
    flat_longitude = layers.Dense(32,activation='relu')(input_longitude)
    flat_other = layers.Dense(64,activation='relu')(input_other)

    # Combine all outputs
    combined = layers.concatenate([
        model_3d.output, 
        model_radial.output, 
        model_cnn.output,
        flat_latitude, 
        flat_longitude, 
        flat_other
    ])

    # Add dense layers for final processing
    x = layers.Dense(128, activation='relu')(combined)  
    x = layers.Dense(1, activation=None)(x)

    # Create the final model
    final_model = models.Model(
        inputs=[model_3d.input, model_radial.input, model_cnn.input,
                input_latitude, input_longitude, input_other ],
        outputs=x
    )

    return final_model

# Build and summarize the updated model
# final_model = build_combined_model()
# final_model.summary()


# Reinitializing the Model

In [6]:
import tensorflow as tf
from keras import backend as K

# Clear session to remove any previously created computation graphs
K.clear_session()
tf.keras.backend.clear_session()

# Re-initialize your model
final_model = build_combined_model()  # Make sure this function initializes your model properly


(None, 48, 48, 8)
(None, 48, 48, 8)
(None, 48, 48, 8)
(None, 48, 48, 8)


# Compiling the Model

In [7]:
from tensorflow.keras import layers, models, optimizers

In [8]:
final_model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                    loss='mse',  # Use 'categorical_crossentropy' for multi-class
                    metrics=['accuracy'])

# Loading our Extracted data from TCIR dataset

In [9]:
import numpy as np
reduced_images=np.load('/kaggle/input/project-data-set/reduced_images.npy')
hov_m_train=np.load('/kaggle/input/project-data-set/hov_m_train.npy')
train_vmax_3d=np.load('/kaggle/input/project-data-set/train_vmax_3d.npy')
lat_train=np.load('/kaggle/input/project-data-set/lat_train.npy')
lon_train=np.load('/kaggle/input/project-data-set/lon_train.npy')
int_diff_train=np.load('/kaggle/input/project-data-set/int_diff_train.npy')
y_train_avg=np.load('/kaggle/input/project-data-set/y_train_avg.npy')
reduced_images_test=np.load('/kaggle/input/project-data-set/reduced_images_test.npy')
hov_m_test=np.load('/kaggle/input/project-data-set/hov_m_test.npy')
test_vmax_3d=np.load('/kaggle/input/project-data-set/test_vmax_3d.npy')
lat_test=np.load('/kaggle/input/project-data-set/lat_test.npy')
lon_test=np.load('/kaggle/input/project-data-set/lon_test.npy')
int_diff_test=np.load('/kaggle/input/project-data-set/int_diff_test.npy')
y_test_avg=np.load('/kaggle/input/project-data-set/y_test_avg.npy')
reduced_images_valid=np.load('/kaggle/input/project-data-set/reduced_images_valid.npy')
hov_m_valid=np.load('/kaggle/input/project-data-set/hov_m_valid.npy')
valid_vmax_3d=np.load('/kaggle/input/project-data-set/valid_vmax_3d.npy')
lat_valid=np.load('/kaggle/input/project-data-set/lat_valid.npy')
lon_valid=np.load('/kaggle/input/project-data-set/lon_valid.npy')
int_diff_valid=np.load('/kaggle/input/project-data-set/int_diff_valid.npy')
y_valid_avg=np.load('/kaggle/input/project-data-set/y_valid_avg.npy')


# Fitting the Model and Prediction

In [10]:
final_model.fit(
    [reduced_images, hov_m_train, train_vmax_3d, lat_train, lon_train, int_diff_train], 
    y_train_avg, 
    validation_data=(
        [reduced_images_valid, hov_m_valid, valid_vmax_3d, lat_valid, lon_valid, int_diff_valid], 
        y_valid_avg
    ),
    epochs=1, 
    batch_size=16
)


311/311 ━━━━━━━━━━━━━━━━━━━━ 95s 219ms/step - accuracy: 0.0000e+00 - loss: 6891.0757 - val_accuracy: 0.0000e+00 - val_loss: 99.7596


In [11]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow Version: 2.17.1
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
final_model.save("convgru-model.h5")


In [13]:
y=final_model.predict([reduced_images_test,hov_m_test,test_vmax_3d,lat_test,lon_test,int_diff_test ])

30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 144ms/step


# Model Evaluation

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [15]:
mae = mean_absolute_error(y_test_avg, y)

# Output the Mean Absolute Error
mae

7.313749315374989

In [16]:
rmse = mean_squared_error(y_test_avg, y, squared=False)

# Output the Root Mean Square Error
print(rmse)

10.302975975339479
